In [1]:
# If you call encode_FE_partial, or encode_BE_partial on training data then the function 
# returns a 2 element python list containing [list, dictionary]
# the return[0] = list are the names of new columns added
# the return[1] = dictionary are which category variables got encoded
# When encoding test data after one of 4 calls above, use 'encode_?E_test'
# and pass the dictionary. If you don't use one of 4 above, then you can
# call basic 'encode_?E' on test.

def nan_check(x):
    if isinstance(x,float):
        if math.isnan(x):
            return True
    return False


# FREQUENCY ENCODER COMBINING TRAIN AND TEST SET
def encode_FE2(df1, df2, col):
    df = pd.concat([df1[col],df2[col]])
    vc = df.value_counts(dropna=False, normalize=True).to_dict()
    nm = col+'_FE'
    df1[nm] = df1[col].map(vc)
    df1[nm] = df1[nm].astype('float32')
    df2[nm] = df2[col].map(vc)
    df2[nm] = df2[nm].astype('float32')

# FREQUENCY ENCODING
def encode_FE(df,col):
    d = df[col].value_counts(dropna=False)
    n = col+"_FE"
    df[n] = df[col].map(d)/d.max()
    return [[n],d]

# BINARY ENCODING
def encode_BE(df,col,val='xyz'):
    if val=='xyz':
        print('BE_encoding all')
        v = df[col].unique()
        n = []
        for x in v: n.append(encode_BE(df,col,x)[0][0])
        return [n,0]
    n = col+"_BE_"+str(val)
    if nan_check(val):
        df[n] = df[col].isna()
    elif isinstance(val, (list,)):
        if not isinstance(val[0], str):
            print('BE_encode Warning: val list not str')
        n = col+"_BE_"+str(val[0])+"_"+str(val[-1])
        d = {}
        for x in val: d[x]=1
        df[n] = df[col].map(d).fillna(0)
    else:
        if not isinstance(val, str):
            print('BE_encode Warning: val is not str')
        df[n] = df[col]==val
    df[n] = df[n].astype('int8')
    return [[n],0]

In [2]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
from datetime import datetime


df_train = pd.read_csv('./train_extra_13.csv') #, usecols=dtypes.keys(), dtype=dtypes)
print ('Loaded',len(df_train),'rows of TRAIN.CSV!')
df_test = pd.read_csv('./test_extra_13.csv') #, usecols=list(dtypes.keys())[0:-1], dtype=dtypes)
print ('Loaded',len(df_test),'rows of TEST.CSV!')

/Users/stathis/anaconda3/envs/dme/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loaded 2230371 rows of TRAIN.CSV!
Loaded 334556 rows of TEST.CSV!


In [5]:
cols = []

# BINARY ENCODE
encode_BE(df_train,'SmartScreen', 'ExistsNotSet')[0]
# encode_BE(df_train,'CountryIdentifier',['89', '214'])[0]
encode_BE(df_train,'Processor','x86')[0]
encode_BE(df_train,'Census_IsTouchEnabled','1')[0]
encode_BE(df_train,'Census_HasOpticalDiskDrive','1')[0]
encode_BE(df_train,'Census_InternalPrimaryDisplayResolutionVertical',['800','600'])[0]
encode_BE(df_train,'Census_PowerPlatformRoleName','Slate')[0]
encode_BE(df_train,'Wdft_IsGamer','1')[0]
encode_BE(df_train,'Census_MDC2FormFactor',['Detachable', 'LargeTablet'])[0]
encode_BE(df_train,'Census_OSWUAutoUpdateOptionsName','FullAuto')[0]

fe_columns = ['AVProductsInstalled',  'CountryIdentifier',  'Census_OSInstallTypeName', 'OsBuild',
 'EngineVersion',  'OsPlatformSubRelease',  'IeVerIdentifier', 'Firewall', 
 'Census_PrimaryDiskTypeName', 'Census_TotalPhysicalRAM', 'Census_OSVersion', 
 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSBuildRevision',  
 'Census_FlightRing',  'Census_FirmwareManufacturerIdentifier', 'Wdft_RegionIdentifier',
 'Census_ProcessorModelIdentifier']

#FREQUENCY ENCODE
for c in fe_columns:
    encode_FE2(df_train, df_test, c)

In [6]:
cols = []

# BINARY ENCODE
encode_BE(df_test,'SmartScreen', 'ExistsNotSet')[0]
encode_BE(df_test,'CountryIdentifier',['89', '214'])[0]
encode_BE(df_test,'Processor','x86')[0]
encode_BE(df_test,'Census_IsTouchEnabled','1')[0]
encode_BE(df_test,'Census_HasOpticalDiskDrive','1')[0]
encode_BE(df_test,'Census_InternalPrimaryDisplayResolutionVertical',['800','600'])[0]
encode_BE(df_test,'Census_PowerPlatformRoleName','Slate')[0]
encode_BE(df_test,'Wdft_IsGamer','1')[0]
encode_BE(df_test,'Census_MDC2FormFactor',['Detachable', 'LargeTablet'])[0]
encode_BE(df_test,'Census_OSWUAutoUpdateOptionsName','FullAuto')[0]

# fe_columns = ['AVProductsInstalled',  'CountryIdentifier',  'Census_OSInstallTypeName', 'OsBuild',
#  'EngineVersion',  'OsPlatformSubRelease',  'IeVerIdentifier', 'Firewall', 
#  'Census_PrimaryDiskTypeName', 'Census_TotalPhysicalRAM', 'Census_OSVersion', 
#  'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSBuildRevision',  
#  'Census_FlightRing',  'Census_FirmwareManufacturerIdentifier', 'Wdft_RegionIdentifier',
#  'Census_ProcessorModelIdentifier']

# #FREQUENCY ENCODE
# for c in fe_columns:
#     encode_FE2(df_test,c)[0]

['Census_OSWUAutoUpdateOptionsName_BE_FullAuto']

In [7]:
df_train.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,...,IeVerIdentifier_FE,Firewall_FE,Census_PrimaryDiskTypeName_FE,Census_TotalPhysicalRAM_FE,Census_OSBranch_FE,Census_OSBuildNumber_FE,Census_FlightRing_FE,Census_FirmwareManufacturerIdentifier_FE,Wdft_RegionIdentifier_FE,Census_ProcessorModelIdentifier_FE
0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.94.0,7.0,0,0.0,53447.0,1.0,1.0,...,0.015687,0.968808,0.658185,0.461596,0.091908,0.091906,0.935220,0.137982,0.141633,0.019037
1,win8defender,1.1.15200.1,4.18.1807.18075,1.275.59.0,7.0,0,0.0,53447.0,1.0,1.0,...,0.413835,0.968808,0.269167,0.241791,0.427200,0.427174,0.935220,0.299664,0.150943,0.001711
2,win8defender,1.1.15200.1,4.18.1807.18075,1.275.180.0,7.0,0,0.0,53447.0,1.0,1.0,...,0.206968,0.968808,0.658185,0.045004,0.142200,0.285876,0.935220,0.008750,0.141633,0.000106
3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.463.0,7.0,0,0.0,53447.0,1.0,1.0,...,0.413835,0.968808,0.658185,0.461596,0.427200,0.427174,0.935220,0.105349,0.207548,0.011578
4,win8defender,1.1.14104.0,4.12.16299.15,1.251.42.0,7.0,0,239.0,53447.0,1.0,1.0,...,0.054017,0.968808,0.658185,0.241791,0.142200,0.285876,0.027787,0.011534,0.140643,0.003385


In [10]:
len(df_train.columns)

102

In [8]:
for c  in df_train.columns:
    print(c)

ProductName
EngineVersion
AppVersion
AvSigVersion
RtpStateBitfield
IsSxsPassiveMode
DefaultBrowsersIdentifier
AVProductStatesIdentifier
AVProductsInstalled
AVProductsEnabled
HasTpm
CountryIdentifier
CityIdentifier
OrganizationIdentifier
GeoNameIdentifier
LocaleEnglishNameIdentifier
Processor
OsVer
OsBuild
OsSuite
OsPlatformSubRelease
OsBuildLab
SkuEdition
IsProtected
IeVerIdentifier
SmartScreen
Firewall
Census_MDC2FormFactor
Census_OEMNameIdentifier
Census_OEMModelIdentifier
Census_ProcessorCoreCount
Census_ProcessorManufacturerIdentifier
Census_ProcessorModelIdentifier
Census_PrimaryDiskTotalCapacity
Census_PrimaryDiskTypeName
Census_SystemVolumeTotalCapacity
Census_HasOpticalDiskDrive
Census_TotalPhysicalRAM
Census_ChassisTypeName
Census_InternalPrimaryDiagonalDisplaySizeInInches
Census_InternalPrimaryDisplayResolutionHorizontal
Census_InternalPrimaryDisplayResolutionVertical
Census_PowerPlatformRoleName
Census_InternalBatteryType
Census_InternalBatteryNumberOfCharges
Census_OSVersio

In [9]:
export_csv = df_train.to_csv(r'train_ffe_comb.csv', index=None, header=True)
export_csv = df_test.to_csv(r'test_ffe_comb.csv', index=None, header=True)